In [11]:
import numpy as np
import re
import pandas as pd

In [3]:
def generate_bool_template(n):
    size = 2 ** n
    arr = []
    for i in range(1, n + 1):
        row = []
        counter = 0
        step = int(size / (2 ** i))
        current = True
        while counter < size:
            for j in range(step):
                row.append(current)
                counter += 1
            current = not current
        arr.append(row)
    result = (np.array(arr)).T
    return result


In [15]:
class Equ:
    operations = {'brackets': ['(', ')'],
    'binary':['&', '|'],
    'unary':['!']}
    priorities = {'!':1,
    '&':2,
    '|':3,
    '(':4,
    ')':4}
    def __init__(self, equation):
        self.equation = equation
        self.variables = Equ.parse_variables(equation)
        self.unique_variables = list(dict.fromkeys(self.variables))
        unique_variables_matrix = Equ.generate_bool_template(len(self.unique_variables))
        self.variables_matrix = unique_variables_matrix.copy()
        self.dict_vars = {}
        Equ.initialize_variables_matrixes(self, unique_variables_matrix)
        #create dict of variables
        self.vars_in_eq = {}
        Equ.create_dict_of_variables(self)
        #to exit list
        self.exit_list = Equ.toRPN(equation)
        Equ.solve(self)
    def solve(self):
        i = 0
        next_name = 0
        while i < len(self.exit_list):
            val = self.exit_list[i]
            if Equ.isoperation(val):
                if Equ.isbinary_op(val):
                    #взять два предыдущих элемента (переменных) в стеке и найти их Variables в self.vars_in_eq
                    first_v = self.vars_in_eq[self.exit_list[i-2]]
                    second_v = self.vars_in_eq[self.exit_list[i-1]]
                    #вычислить операцию, выкинуть элементы и операцию из exit, записать имя новой переменно в exit и саму переменную в vars_in_eq
                    res = Equ.calculate_binary(first_v, second_v, val, str(next_name))
                    del (self.exit_list[i-2:i+1])
                    self.exit_list.insert(i-2, str(next_name))
                    self.vars_in_eq[str(next_name)] = res
                    i -= 1
                    next_name += 1 
                elif Equ.isunary_op(val):
                    #взять 1 предыдущий элемент (переменную) в стеке и найти его Variable в self.vars_in_eq
                    this_v = self.vars_in_eq[self.exit_list[i-1]]
                    #вычислить операцию, выкинуть элемент и операцию из exit, записать имя новой переменно в exit и саму переменную в vars_in_eq
                    res = Equ.calculate_unary(this_v, val, str(next_name))
                    del (self.exit_list[i-1:i+1])
                    self.exit_list.insert(i-1, str(next_name))
                    self.vars_in_eq[str(next_name)] = res
                    #здесь i не меняется
                    next_name += 1
            else:
                i += 1
            print(self.exit_list)
    def initialize_variables_matrixes(self, unique_variables_matrix):
        for i in self.variables:
            for j in range(len(self.unique_variables)):
                if i == self.unique_variables[j]:
                    self.dict_vars[i] = j
                    break
        #initialize full variables_matrix
        for i in range(len(self.variables_matrix[0]), len(self.variables)):
            num_of_column = self.dict_vars[self.variables[i]]
            column = unique_variables_matrix[:,num_of_column].copy()
            self.variables_matrix = np.append(self.variables_matrix, column.reshape(len(column), 1), axis=1)
    def create_dict_of_variables(self):
        for i in range(len(self.variables)):
            self.vars_in_eq[self.variables[i]] = Variable(name=self.variables[i], column=self.variables_matrix[:,i].copy())
    def get_result(self):
        return pd.DataFrame(self.vars_in_eq[self.exit_list[0]].column)
    @staticmethod
    def calculate_binary(var1, var2, op, name):
        if op == '&':
            var3 = var1 & var2
            return Variable(name=name, column=var3.column)
        elif op == '|':
            var3 = var1 | var2
            return Variable(name=name, column=var3.column)
        else:
            raise ValueError("Couldnt find this operations. Try & or |")
    @staticmethod
    def calculate_unary(var, op, name):
        if op == '!':
            var1 = - var
            return Variable(name=name, column=var1.column)
        else:
            raise ValueError("Couldnt find this operations. Try !")
    @staticmethod
    def toRPN(equation):
        exit_list = []
        stack = []
        #просто следуем алгоритму
        for i in equation:
            #если переменная, добавляем к выходу
            if i.isalpha():
                exit_list.append(i)
            #если префиксная функция, добавляем к стеку
            elif Equ.isprefix_func(i):
                stack.append(i)
            #если открывающая скобка, добавляем к стеку
            elif Equ.isopening_bracket(i):
                stack.append(i)
            #если закрывающая скобка, выталкиваем их стека
            #всё до открывающей и помещаем в выход
            elif Equ.isclosing_bracket(i):
                try:
                    while not Equ.isopening_bracket(stack[-1]):
                        exit_list.append(stack.pop())
                    stack.pop()
                except IndexError:
                    print("Too many brackets in the equation! Check ot out!")
            elif Equ.isbinary_op(i):
                while (len(stack) > 0) and (Equ.isprefix_func(stack[-1]) or \
                Equ.priorities[stack[-1]] < Equ.priorities[i] or \
                Equ.priorities[stack[-1]] == Equ.priorities[i]):
                    exit_list.append(stack.pop())
                stack.append(i)
        #строка закончилась, выталкиваем элементы из стека
        while len(stack) > 0:
            exit_list.append(stack.pop())
        return exit_list
                
    @staticmethod
    def isprefix_func(func):
        return func == '!'
    @staticmethod
    def isopening_bracket(symbol):
        return symbol == '('
    @staticmethod
    def isclosing_bracket(symbol):
        return symbol == ')'
    @staticmethod
    def isbinary_op(op):
        return op in Equ.operations['binary']
    @staticmethod
    def isunary_op(op):
        return op in Equ.operations['unary']
    @staticmethod
    def isoperation(symbol):
        return Equ.isbinary_op(symbol) or Equ.isunary_op(symbol)
    @staticmethod
    def parse_variables(equation):
        variables = []
        for char in equation:
            if char.isalpha():
                variables.append(char)
        return variables
    @staticmethod
    def generate_bool_template(n):
        size = 2 ** n
        arr = []
        for i in range(1, n + 1):
            row = []
            counter = 0
            step = int(size / (2 ** i))
            current = True
            while counter < size:
                for j in range(step):
                    row.append(current)
                    counter += 1
                current = not current
            arr.append(row)
        result = (np.array(arr)).T
        return result
    def result(self):
        return self.result
class Variable:
    def __init__(self, name, column):
        self.name = name
        self.column = column
    def __neg__(self):
        #Overload -
        new_name = "n" + self.name
        new_column = self.column.copy()
        for i in range(len(self.column)):
                new_column[i] = not self.column[i]
        return Variable(new_name, new_column)
    def __not__(self):
        new_name = "n" + self.name
        new_column = self.column
        for i in range(len(self.column)):
                new_column[i] = not self.column[i]
                print("changed")
        return Variable(new_name, new_column)
    def __str__(self):
        return str(self.column)
    def __repr__(self):
        return str(self.column)
    def __or__(self, other):
        #overload |
        new_name = self.name
        new_column = self.column.copy()
        for i in range(len(self.column)):
            new_column[i] = new_column[i] or other.column[i]
        return Variable(new_name, new_column)
    def __and__(self, other):
        #overload &
        new_name = self.name
        new_column = self.column.copy()
        for i in range(len(self.column)):
            new_column[i] = new_column[i] and other.column[i]
        return Variable(new_name, new_column)

In [16]:
eq = "((A&!A)|(B|!B)|(C|D))&E"
new_eq = Equ(eq)
result = new_eq.get_result()
result

['A', 'A', '!', '&', 'B', 'B', '!', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['A', 'A', '!', '&', 'B', 'B', '!', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['A', '0', '&', 'B', 'B', '!', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['1', 'B', 'B', '!', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['1', 'B', 'B', '!', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['1', 'B', 'B', '!', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['1', 'B', '2', '|', '|', 'C', 'D', '|', '|', 'E', '&']
['1', '3', '|', 'C', 'D', '|', '|', 'E', '&']
['4', 'C', 'D', '|', '|', 'E', '&']
['4', 'C', 'D', '|', '|', 'E', '&']
['4', 'C', 'D', '|', '|', 'E', '&']
['4', '5', '|', 'E', '&']
['6', 'E', '&']
['6', 'E', '&']
['7']


,0
0,True
1,False
2,True
3,False
4,True
5,False
6,True
7,False
8,True
9,False


In [49]:
A = Variable('A', np.array([True, True, False, False]))
print(A)
B = - A
print(B)
print(A)
C = (A|B)&(-A)
print(C)

[ True  True False False]
[False False  True  True]
[ True  True False False]
[False False  True  True]
